In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import polars as pl
import geopandas as gpd
import h5py
import util

%matplotlib inline
from IPython.display import display, HTML

relative_path = Path('../../../..')

pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:0,.0f}'.format

## Transit Boardings

In [2]:

df = pd.read_csv(util.output_path / 'transit/daily_boardings_by_agency.csv',index_col=0)
df.loc['Region Total','boardings'] = df['boardings'].sum()
df = df.reset_index()
df.rename(columns={'agency_name': 'Agency', 'boardings': 'Boardings'}, inplace=True)
HTML(df.to_html(index=False))

Agency,Boardings
King County Metro,"778,445"
Sound Transit,"512,202"
Community Transit,"99,474"
Pierce Transit,"71,625"
Kitsap Transit,"43,100"
Washington Ferries,"23,421"
Everett Transit,"17,889"
Region Total,"1,546,156"


In [3]:
df = pd.read_csv(util.output_path / 'transit/transit_line_results.csv',index_col=0)

transit_type_map = {
    1: 'Local Bus',
    2: 'Express Bus',
    3: 'Bus Rapid Transit',
    4: 'Streetcar',
    5: 'Commuter Rail',
    6: 'Light Rail',
    7: 'Auto Ferry',
    8: 'Passenger Ferry'
}
df['transit_type'] = df['transit_type'].map(transit_type_map)
df = df.groupby('transit_type').sum()[['boardings']]
df.loc['Total'] = df.sum(axis=0)
df = df.reset_index()
df.rename(columns={'boardings':' Boardings','transit_type':'Transit Type'}, inplace=True)
HTML(df.to_html(index=False))

Transit Type,Boardings
Auto Ferry,"36,857"
Bus Rapid Transit,"391,067"
Commuter Rail,"9,520"
Light Rail,"471,412"
Local Bus,"617,460"
Passenger Ferry,"10,280"
Streetcar,"9,561"
Total,"1,546,156"


In [4]:
df = pd.read_csv(util.output_path / 'transit/transit_line_results.csv',index_col=0)
df = df[(df['agency_code']==6)].groupby('transit_type').sum()[['boardings']]
df.index = df.index.map({1: 'Bus', 3: 'BRT (Stride)', 5: 'Commuter Rail', 6: 'Light Rail'})
df.rename(columns={'boardings':' Boardings'}, inplace=True)

df

,Boardings
transit_type,
Bus,"7,959"
BRT (Stride),"23,311"
Commuter Rail,"9,520"
Light Rail,"471,412"


In [5]:
# all transit stop locations
df_transit_stops = pd.read_csv(util.input_path / 'scenario/networks/transit_stops.csv', usecols = ['PSRCJunctID', 'x', 'y','light_rail'])

# Load station names
df_lr_station_names = util.read_sqlite_db("SELECT * FROM light_rail_stations_2050")

# Merge station names to df 
light_rail_stations = df_transit_stops[df_transit_stops['light_rail']==1].\
    merge(df_lr_station_names, left_on=['PSRCJunctID'], right_on=['Station Node'])

In [6]:
# Light rail boardings by station
df_boardings = pd.read_csv(util.output_path / 'transit/boardings_by_stop.csv')
df = pd.merge(df_boardings, light_rail_stations, left_on=['i_node'], right_on=['PSRCJunctID'])

df = df[['total_boardings','Station Name','PSRCJunctID']]#.drop_duplicates()

# pivot table of boardings by station 
df[['PSRCJunctID','Station Name','total_boardings']]

,PSRCJunctID,Station Name,total_boardings
0,197649,Husky Stadium,"11,765"
1,198007,Westlake,"33,160"
2,198009,Symphony,"18,928"
3,198011,Pioneer Square,"16,397"
4,198013,Int'l District,"26,615"
5,198014,Capitol Hill,"16,145"
6,198015,Stadium,"2,667"
7,198016,SODO,"13,734"
8,198017,Beacon Hill,"3,230"
9,198018,Mount Baker,"3,356"


In [7]:
# Light rail boardings by segment
df = pd.read_csv(util.output_path / 'transit/transit_line_results.csv', usecols = ['line_id','mode','description'])
df_light_rail = df[(df['mode']=="r") & (df['description']!="Streetcar-South Lake")].copy().\
    drop_duplicates()
df_light_rail['description'] = df_light_rail['description'].str[:6]
df_light_rail.rename(columns={'description':'Line'}, inplace=True)

In [8]:
# Light rail boardings by segment
df_boardings = pd.read_csv(util.output_path / 'transit/transit_segment_results.csv')

# get line and depart station names
df = df_boardings.merge(df_lr_station_names, left_on=['i_node'], right_on=['Station Node']).\
    merge(df_light_rail, on=['line_id'])

# all station boardings by line
board_station_line = df.groupby(['Line','Station Name'])['segment_boarding'].sum().reset_index()

In [9]:
board_station_line.groupby('Line')['segment_boarding'].sum().\
    reset_index().set_index('Line').\
    rename(columns={'segment_boarding':'Total Boardings'})

,Total Boardings
Line,
1 Line,"126,866"
2 Line,"171,637"
3 Line,"138,959"
4 Line,"20,184"
T Line,"13,767"


In [10]:
def get_stop_boardings(line_name):
    df = board_station_line[board_station_line['Line']==line_name]
    df2 = df.groupby('Line').sum()['segment_boarding'].reset_index()
    df2['Station Name'] = 'Total'
    return pd.concat([df2[["Line", "Station Name", "segment_boarding"]],df], ignore_index=True)

get_stop_boardings('1 Line')

,Line,Station Name,segment_boarding
0,1 Line,Total,"126,866"
1,1 Line,Angle Lake,"4,069"
2,1 Line,Ballard Station,"5,627"
3,1 Line,Beacon Hill,"3,230"
4,1 Line,Boeing Access Station,5
5,1 Line,Columbia City,"2,231"
6,1 Line,Denny Station,"5,062"
7,1 Line,East Tacoma Station,"2,398"
8,1 Line,Federal Way Station,"7,898"
9,1 Line,Fife Station,"2,535"


In [11]:
get_stop_boardings('2 Line')

,Line,Station Name,segment_boarding
0,2 Line,Total,"171,637"
1,2 Line,Alderwood Station,"2,907"
2,2 Line,Ash Way Station,"2,929"
3,2 Line,BelRed Station,"5,176"
4,2 Line,Bellevue Downtown Station,"12,036"
5,2 Line,Capitol Hill,"8,211"
6,2 Line,Downtown Redmond Station,"4,993"
7,2 Line,East Main Station,"8,581"
8,2 Line,Husky Stadium,"5,518"
9,2 Line,Int'l District,"10,440"


In [12]:
get_stop_boardings('3 Line')

,Line,Station Name,segment_boarding
0,3 Line,Total,"138,959"
1,3 Line,Alderwood Station,"3,255"
2,3 Line,Ash Way Station,"3,660"
3,3 Line,Avalon Station,"2,607"
4,3 Line,Capitol Hill,"7,934"
5,3 Line,Delridge Station,"6,301"
6,3 Line,Everett Station,"6,197"
7,3 Line,Husky Stadium,"6,247"
8,3 Line,Int'l District,"6,465"
9,3 Line,Lynnwood Station,"5,778"


In [13]:
get_stop_boardings('4 Line')

,Line,Station Name,segment_boarding
0,4 Line,Total,"20,184"
1,4 Line,Bellevue Downtown Station,"4,573"
2,4 Line,East Main Station,"4,796"
3,4 Line,Eastgate Station,"2,672"
4,4 Line,Issaquah Station,"2,996"
5,4 Line,Richards Rd Station,"2,561"
6,4 Line,South Kirkland Station,"1,273"
7,4 Line,Wilburton Station,"1,313"


In [14]:
get_stop_boardings('T Line')

,Line,Station Name,segment_boarding
0,T Line,Total,"13,767"
1,T Line,6th Ave Station,"1,395"
2,T Line,Ainsworth Station,455
3,T Line,Commerce Station,470
4,T Line,Convention Center Station,285
5,T Line,Hilltop District Station,499
6,T Line,Old City Hall Station,357
7,T Line,Pearl Station,825
8,T Line,S 25th St Station,989
9,T Line,S 4th St Station,421


In [15]:
df = pd.read_csv(util.output_path / 'transit/total_transit_trips.csv',index_col=0)
df.rename(index={'commuter_rail': 'Commuter Rail',
                'litrat': 'Light Rail',
                'ferry': 'Auto Ferry',
                'passenger_ferry': 'Passenger Ferry',
                'trnst': 'Bus'}, inplace=True)
df.columns = ['Total Trips']
df.loc['Total'] = df.sum(axis=0)
df.reset_index(inplace=True)
df.rename(columns={'index': 'Transit Type'}, inplace=True)
HTML(df.to_html(index=False))

Transit Type,Total Trips
Commuter Rail,"10,442"
Light Rail,"438,157"
Auto Ferry,"40,155"
Passenger Ferry,"4,851"
Bus,"561,248"
Total,"1,054,853"


## Households Near HCT

In [16]:
# List of Stations
# Load transit stops file

#"The definition we have been using for the RTP is BRT, LRT, Commuter Rail, StreetCar and Ferry."
df = pd.read_csv(util.input_path / 'scenario/networks/transit_stops.csv')
# Streetcar is coded as light rail
df['hct'] = 0
df.loc[df[['commuter_rail','light_rail','ferry','brt']].sum(axis=1) > 0,'hct'] = 1
df_hct = df[df['hct'] == 1]

# Map of Stations
# Load as a geodataframe
gdf_hct = gpd.GeoDataFrame(
    df_hct, geometry=gpd.points_from_xy(df_hct.x, df_hct.y))

gdf_hct.crs = 'EPSG:2285'

In [17]:
df_lu = util.get_parcels_urbansim_data(inc_geog=True)
parcel_geog = util.get_parcel_geog()

# Load as a geodataframe
gdf_lu = gpd.GeoDataFrame(
    df_lu, geometry=gpd.points_from_xy(df_lu.xcoord_p, df_lu.ycoord_p))

gdf_lu.crs = 'EPSG:2285'

In [18]:
# Add a field that defines whether a parcel is inside (1) or outside (0) any RGC
df_lu['RGC_binary'] = 0
df_lu.loc[df_lu['GrowthCenterName'] != 'Not in RGC', 'RGC_binary'] = 1

In [19]:
def calculate_buffer(gdf_lu, _gdf_hct, distance):
    
    # Buffer the HCT station gdf
    _gdf_hct['geometry'] = _gdf_hct.buffer(distance)

    gdf_intersect = gpd.overlay(_gdf_hct, gdf_lu, how="intersection", keep_geom_type=False)
    df = df_lu[df_lu['parcelid'].isin(gdf_intersect['parcelid'].unique())]
    
    return df, gdf_intersect

def aggregate_parcels(df, col_dict, sum_field):
    results_df = pd.DataFrame()
    for col, name in col_dict.items():
        _df = df[[col,sum_field]].groupby(col).sum()[[sum_field]]
        _df['group'] = name
        results_df = pd.concat([results_df,_df])
    results_df = results_df.reset_index()
    
    return results_df

In [20]:
df_025, gdf_025 = calculate_buffer(gdf_lu, gdf_hct.copy(), distance=5280.0/4)

In [21]:
df_050, gdf_050 = calculate_buffer(gdf_lu, gdf_hct.copy(), distance=5280.0/2)

In [22]:
pd.options.display.float_format = '{:0,.0f}'.format
col_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability',
           'rg_proposed': 'Regional Geography',
            'CountyName': 'County',
            'GrowthCenterName': 'Regional Growth Center',
            'RGC_binary': 'RGC Binary',
            'Region': 'Region'
           }

hct_hh_df = pd.DataFrame()
df_025 = df_025.copy()
df_025['Region'] = 1
df = aggregate_parcels(df_025, col_dict, 'hh_p')
df['distance'] = 0.25
hct_hh_df = pd.concat([hct_hh_df,df])
df_050 = df_050.copy()
df_050['Region'] = 1
df = aggregate_parcels(df_050, col_dict, 'hh_p')
df['distance'] = 0.50
hct_hh_df = pd.concat([hct_hh_df,df])

In [23]:
pd.options.display.float_format = '{:0,.0f}'.format

In [24]:

df = hct_hh_df[hct_hh_df['group'] == 'Region']
df = df.rename(columns={'hh_p': 'Total'}).copy()
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Region'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df = df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'})

df['% of Households (1/4 Mile)'] = df['1/4 Mile']/df_lu['hh_p'].sum()
df['% of Households (1/2 Mile)'] = df['1/2 Mile']/df_lu['hh_p'].sum()

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Region,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
1,"786,248","1,146,975",32.5%,47.4%


In [25]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'County'].copy()
df = df.rename(columns={'hh_p': 'Total'}).copy()
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'County'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

_df = df.merge(df_lu[['CountyName','hh_p']].groupby('CountyName').sum()[['hh_p']].reset_index(),left_on='County', right_on='CountyName')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['County', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='County')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

County,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
King,"618,387","826,741",48.0%,64.2%
Kitsap,"17,724","36,500",11.8%,24.3%
Pierce,"46,373","79,884",9.3%,16.0%
Snohomish,"103,764","203,850",21.5%,42.3%


In [26]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'RGC Binary'].copy()
df.rename(columns={'hh_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'RGC Binary'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)


_df = df.merge(df_lu[['RGC_binary','hh_p']].groupby('RGC_binary').sum()[['hh_p']].reset_index(),left_on='RGC Binary', 
               right_on='RGC_binary')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['RGC Binary', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='RGC Binary')
df['RGC Designation'] = df['RGC Binary'].map({0: 'Outside RGC', 1: 'Inside RGC'})
df = df[['RGC Designation','1/4 Mile','1/2 Mile','% of Households (1/4 Mile)', '% of Households (1/2 Mile)']]
output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

RGC Designation,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Outside RGC,"441,853","740,147",22.4%,37.5%
Inside RGC,"344,395","406,828",77.2%,91.3%


In [27]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Growth Center'].copy()
df.rename(columns={'hh_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Growth Center'
# Ensure all RGCs are included, even those that have no HCT access
for center in parcel_geog['GrowthCenterName'].unique():
    if center not in df.index:
        df.loc[center] = 0

df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

# Calcualte the percent of households in this geography within 1/4 and 1/2 mile
_df = df.merge(df_lu[['GrowthCenterName','hh_p']].groupby('GrowthCenterName').sum()[['hh_p']].reset_index(),left_on='Regional Growth Center', 
               right_on='GrowthCenterName')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['Regional Growth Center', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='Regional Growth Center')

df = df.sort_values('Regional Growth Center')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Growth Center,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Auburn,"4,805","5,005",96.0%,100.0%
Bellevue,"32,173","32,904",97.8%,100.0%
Bothell Canyon Park,729,"3,438",13.7%,64.7%
Bremerton,"5,360","5,360",99.5%,99.5%
Burien,"8,570","8,803",97.4%,100.0%
Everett,"11,135","20,877",49.7%,93.2%
Federal Way,"5,949","6,226",95.6%,100.0%
Greater Downtown Kirkland,"5,468","6,157",88.7%,99.9%
Issaquah,"1,196","2,634",45.4%,100.0%
Kent,"3,504","3,854",90.9%,100.0%


In [28]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Geography'].copy()
df.rename(columns={'hh_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Geography'
df.reset_index(inplace=True)

df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
HTML(df.to_html(index=False))

_df = df.merge(df_lu[['rg_proposed','hh_p']].groupby('rg_proposed').sum()[['hh_p']].reset_index(),left_on='Regional Geography', 
               right_on='rg_proposed')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['Regional Geography', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='Regional Geography')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Geography,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Cities and Towns,464,"2,068",0.3%,1.2%
Core Cities,"188,929","297,576",33.5%,52.7%
High Capacity Transit Communities,"105,846","187,408",20.5%,36.4%
Metropolitan Cities,"486,204","646,059",56.9%,75.6%
Rural Areas,"1,198","3,406",0.5%,1.5%
Urban Unincorporated Areas,"3,607","10,458",4.3%,12.6%


In [29]:
pd.options.display.float_format = '{:0,.2f}'.format
# df = hct_hh_df.copy()
# df = df[df['index'] == 1]
# df.rename(columns={'hh_p': 'Total'}, inplace=True)
# df.drop('index', axis=1, inplace=True)
# df = df.pivot_table(index='group', columns='distance', values='Total', aggfunc='sum')
# df[0.25] = df[0.25].astype('float32')
# df[0.5] = df[0.5].astype('float32')
# df.index.name = 'Equity Geography'
# df = df.reset_index()
# df = df.rename_axis(None, axis=1)
# df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

In [30]:
pd.options.display.float_format = '{:0,.1f}'.format

equity_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability'}

df = hct_hh_df[hct_hh_df['group'].isin(equity_dict.values())].copy()
df['EFA Type'] = df['index'].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

df = df.pivot_table(index=['group','EFA Type'], columns='distance', values='hh_p', aggfunc='sum').reset_index()

# Merge with total households to get shares of group
efa_pop_tot = df_lu[equity_dict.keys()].sum().to_dict()

for lu_col, label in equity_dict.items():
    df.loc[df['group'] == label, "Total Pop"] = efa_pop_tot[lu_col]
df['% of Households (1/4 Mile)'] = df[0.25]/df['Total Pop']
df['% of Households (1/2 Mile)'] = df[0.5]/df['Total Pop']

df.rename(columns={'group': 'Group', 0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df = df.rename_axis(None, axis=1)
df.drop('Total Pop', inplace=True, axis=1)
output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Group,EFA Type,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Disability,Above Regional Average,227686,341194,28.8%,43.2%
Disability,Below Regional Average,393924,573663,49.9%,72.6%
Disability,Higher Share of Equity Population,164638,232118,20.8%,29.4%
LEP,Above Regional Average,205824,300153,35.7%,52.0%
LEP,Below Regional Average,374489,546776,64.9%,94.8%
LEP,Higher Share of Equity Population,205935,300046,35.7%,52.0%
Older,Above Regional Average,181179,281356,19.4%,30.1%
Older,Below Regional Average,506762,726798,54.2%,77.8%
Older,Higher Share of Equity Population,98307,138821,10.5%,14.9%
People of Color,Above Regional Average,287387,418244,45.2%,65.8%


## Jobs Near HCT

In [31]:
pd.options.display.float_format = '{:0,.0f}'.format
col_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability',
           'rg_proposed': 'Regional Geography',
            'CountyName': 'County',
            'GrowthCenterName': 'Regional Growth Center',
            'RGC_binary': 'RGC Binary',
            'Region': 'Region'
           }

hct_hh_df = pd.DataFrame()
df_025['Region'] = 1
df = aggregate_parcels(df_025, col_dict, 'emptot_p')
df['distance'] = 0.25
hct_hh_df = pd.concat([hct_hh_df,df])
df_050['Region'] = 1
df = aggregate_parcels(df_050, col_dict, 'emptot_p')
df['distance'] = 0.50
hct_hh_df = pd.concat([hct_hh_df,df])

In [32]:

df = hct_hh_df[hct_hh_df['group'] == 'Region'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Region'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

df['% of Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['emptot_p'].sum()
df['% of Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['emptot_p'].sum()

output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Region,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
1,"1,622,715","2,124,904",51.3%,67.2%


In [33]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'County'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'County'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

_df = df.merge(df_lu[['CountyName','emptot_p']].groupby('CountyName').sum()[['emptot_p']].reset_index(),left_on='County', right_on='CountyName')
_df['% of Jobs (1/4 Mile)'] = _df['1/4 Mile']/_df['emptot_p']
_df['% of Jobs (1/2 Mile)'] = _df['1/2 Mile']/_df['emptot_p']

df = df.merge(_df[['County', '% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']], 
              on='County')

output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

County,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
King,"1,314,754","1,616,217",64.2%,78.9%
Kitsap,"40,863","70,977",27.8%,48.2%
Pierce,"101,940","150,312",21.6%,31.8%
Snohomish,"165,158","287,398",33.5%,58.3%


In [34]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'RGC Binary'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'RGC Binary'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)


_df = df.merge(df_lu[['RGC_binary','emptot_p']].groupby('RGC_binary').sum()[['emptot_p']].reset_index(),left_on='RGC Binary', 
               right_on='RGC_binary')
_df['% of Jobs (1/4 Mile)'] = _df['1/4 Mile']/_df['emptot_p']
_df['% of Jobs (1/2 Mile)'] = _df['1/2 Mile']/_df['emptot_p']

df = df.merge(_df[['RGC Binary', '% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']], 
              on='RGC Binary')
df['RGC Designation'] = df['RGC Binary'].map({0: 'Outside RGC', 1: 'Inside RGC'})
df = df[['RGC Designation','1/4 Mile','1/2 Mile','% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']]
output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

RGC Designation,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
Outside RGC,"641,565","1,003,999",32.9%,51.5%
Inside RGC,"981,150","1,120,905",81.1%,92.6%


In [35]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Growth Center'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Growth Center'
# Ensure all RGCs are included, even those that have no HCT access
for center in parcel_geog['GrowthCenterName'].unique():
    if center not in df.index:
        df.loc[center] = 0
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df['% of Total Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['emptot_p'].sum()
df['% of Total Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['emptot_p'].sum()

output = df.to_html(formatters={
    '% of Total Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Total Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Growth Center,1/4 Mile,1/2 Mile,% of Total Jobs (1/4 Mile),% of Total Jobs (1/2 Mile)
Auburn,"9,039","9,693",0.3%,0.3%
Bellevue,"97,138","98,694",3.1%,3.1%
Bothell Canyon Park,"3,600","11,345",0.1%,0.4%
Bremerton,"8,865","17,996",0.3%,0.6%
Burien,"8,753","8,962",0.3%,0.3%
Everett,"36,484","51,927",1.2%,1.6%
Federal Way,"11,815","12,628",0.4%,0.4%
Greater Downtown Kirkland,"19,743","20,446",0.6%,0.6%
Issaquah,"3,217","14,131",0.1%,0.4%
Kent,"31,480","33,419",1.0%,1.1%


In [36]:
df = hct_hh_df[hct_hh_df['group'] == 'Regional Geography'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Geography'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df['% of Total Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['emptot_p'].sum()
df['% of Total Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['emptot_p'].sum()

output = df.to_html(formatters={
    '% of Total Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Total Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Geography,1/4 Mile,1/2 Mile,% of Total Jobs (1/4 Mile),% of Total Jobs (1/2 Mile)
Cities and Towns,847,"2,414",0.0%,0.1%
Core Cities,"468,755","677,663",14.8%,21.4%
High Capacity Transit Communities,"106,567","165,489",3.4%,5.2%
Metropolitan Cities,"1,034,743","1,258,470",32.7%,39.8%
Rural Areas,"4,474","7,749",0.1%,0.2%
Urban Unincorporated Areas,"7,329","13,119",0.2%,0.4%


In [37]:
pd.options.display.float_format = '{:0,.0f}'.format

equity_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability'}

df = hct_hh_df[hct_hh_df['group'].isin(equity_dict.values())].copy()
df['EFA Type'] = df['index'].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

df = df.pivot_table(index=['group','EFA Type'], columns='distance', values='emptot_p', aggfunc='sum').reset_index()

# Merge with total households to get shares of group
efa_pop_tot = df_lu[equity_dict.keys()].sum().to_dict()

df[0.25] = df[0.25].astype("float")
df[0.50] = df[0.50].astype("float")

for lu_col, label in equity_dict.items():
    df.loc[df['group'] == label, "Total Jobs"] = efa_pop_tot[lu_col]
df['% of Households (1/4 Mile)'] = df[0.25]/df['Total Jobs']
df['% of Households (1/2 Mile)'] = df[0.5]/df['Total Jobs']


df.rename(columns={'group': 'Group', 0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df = df.rename_axis(None, axis=1)
df.drop('Total Jobs', inplace=True, axis=1)
output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Group,EFA Type,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Disability,Above Regional Average,"430,957","573,081",1,1
Disability,Below Regional Average,"741,486","995,431",1,1
Disability,Higher Share of Equity Population,"450,272","556,392",1,1
LEP,Above Regional Average,"460,269","590,890",1,1
LEP,Below Regional Average,"728,657","967,138",1,2
LEP,Higher Share of Equity Population,"433,789","566,876",1,1
Older,Above Regional Average,"401,377","526,418",0,1
Older,Below Regional Average,"1,045,926","1,355,069",1,1
Older,Higher Share of Equity Population,"175,412","243,417",0,0
People of Color,Above Regional Average,"685,737","848,780",1,1


## Transit Access Mode
Percent of Transit Trips Accessed by Walking

In [38]:
pd.options.display.float_format = '{:0,.1%}'.format
df = pd.read_csv(util.output_path / 'agg/dash/trip_mode_by_tour_mode.csv')
_df = df.copy()
pnr_transit_trips = _df[(_df['tmodetp'] == 'Park') & (_df['mode'] == 'Transit')]['trexpfac'].sum()
walk_transit_trips = _df[(_df['tmodetp'] == 'Transit') & (_df['mode'] == 'Transit')]['trexpfac'].sum()
reg_access = (walk_transit_trips/(walk_transit_trips+pnr_transit_trips))
pd.DataFrame([reg_access], columns=['% Transit trips Accessed by Walking'], index=['Region'])


,% Transit trips Accessed by Walking
Region,96.5%


In [39]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = df.copy()
results_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
# for col in ['hh_disability','hh_elderly','hh_english','hh_poverty','hh_racial','hh_youth']:
for col in util.summary_config["hh_equity_geogs"]:
    pnr_transit_trips = _df[(_df[col] == 1) & 
                            (_df['tmodetp'] == 'Park') &
                            (_df['mode'] == 'Transit')]['trexpfac'].sum()
    walk_transit_trips = _df[(_df[col] == 1) & 
                                (_df['tmodetp'] == 'Transit') & 
                                (_df['mode'] == 'Transit')]['trexpfac'].sum()
    if (pnr_transit_trips > 0) & (walk_transit_trips > 0):
        results_df.loc[col,'% Transit Trips Accessed by Walking'] = walk_transit_trips/(walk_transit_trips+pnr_transit_trips)
# results_df.rename(columns={'_reg':'> Regional Average', '_50': '> 50%'}, inplace=True)
results_df

,% Transit Trips Accessed by Walking
hh_efa_dis,96.6%
hh_efa_older,95.9%
hh_efa_lep,96.5%
hh_efa_pov200,96.7%
hh_efa_poc,96.6%
hh_efa_youth,94.9%


## Transit Equity

### Trip purpose by transit mode type

In [40]:
myh5 = h5py.File(util.input_path / 'scenario/landuse/hh_and_persons.h5', 'r')
person = pd.read_csv(util.output_path / 'daysim/_person.tsv', sep='\t')
df_trip = pd.read_csv(util.output_path / 'daysim/_trip.tsv', sep='\t')
df_hh = pd.read_csv(util.output_path / 'daysim/_household.tsv', sep='\t')

In [41]:
# Create a DataFrame from the h5 file
df_person = pd.DataFrame()
for col in ['hhno','pno','prace']:
    df_person[col] = myh5['Person'][col][:]

df_trip = df_trip.merge(df_person[['hhno','pno','prace']], on=['hhno','pno'], how='left')

In [42]:
# Create group of work, school, and all other purposes
df_trip['Purpose Type'] = 'Other'
df_trip.loc[df_trip['dpurp']==0, 'Purpose Type'] = 'Home'
df_trip.loc[df_trip['dpurp']==1, 'Purpose Type'] = 'Work'
df_trip.loc[df_trip['dpurp']==2, 'Purpose Type'] = 'School'

# Rename path types
df_trip.loc[df_trip['pathtype']==1, 'Path Type'] = 'Drive'
df_trip.loc[df_trip['pathtype']==3, 'Path Type'] = 'Bus'
df_trip.loc[df_trip['pathtype']==4, 'Path Type'] = 'Light Rail'
df_trip.loc[df_trip['pathtype']==5, 'Path Type'] = 'Ferry 1'
df_trip.loc[df_trip['pathtype']==6, 'Path Type'] = 'Commuter Rail'
df_trip.loc[df_trip['pathtype']==7, 'Path Type'] = 'Ferry 2'

df = pd.pivot_table(df_trip[(df_trip['dpurp']!=0) & (~df_trip['pathtype'].isin([0,'Drive']))], values='trexpfac', index='Path Type', columns='Purpose Type', aggfunc='sum')

df['Total'] = df.sum(axis=1)
for row in df.index:
    for col in ['Work','School','Other']:
        df.loc[row, col + ' %'] = df.loc[row, col] / df.loc[row, 'Total']

df[['Work %','School %','Other %']] = df[['Work %','School %','Other %']].map("{:,.1%}".format)
df[['Work %','School %','Other %']]

Purpose Type,Work %,School %,Other %
Path Type,,,
Bus,14.3%,12.9%,72.8%
Commuter Rail,50.7%,2.7%,46.5%
Drive,20.6%,5.5%,73.9%
Ferry 1,65.7%,6.1%,28.1%
Ferry 2,52.6%,8.3%,39.1%
Light Rail,31.4%,12.7%,56.0%


In [43]:

race_dict = {
    '1': 'White alone',
    '2': 'Black or African American alone',
    '3': 'American Indian alone',
    '4': 'Alaska Native alone',
    '5': 'American Indian and Alaska Native tribes specified',
    '6': 'Asian alone',
    '7': 'Native Hawaiian and Other Pacific Islander alone',
    '8': 'Some Other Race alone',
    '9': 'Two or More Races'
}


df_trip['Race'] = df_trip['prace'].astype('int').astype('str').map(race_dict)

### Transit type distribution by race of rider

In [44]:
# Submode Ridership by race
df = pd.pivot_table(df_trip[(df_trip['pathtype']!="Drive")], values='trexpfac', index='Path Type', columns='Race', aggfunc="sum")
race_cols = ['White alone','Black or African American alone','American Indian alone',
             'Alaska Native alone','American Indian and Alaska Native tribes specified','Asian alone',
             'Native Hawaiian and Other Pacific Islander alone','Some Other Race alone','Two or More Races']
# Calculate shares of total by Race for each Path Type
df['Total'] = df.sum(axis=1)
df = df.astype('float')
for row in df.index:
    for col in race_cols:
        df.loc[row, col] = df.loc[row, col] / df.loc[row, 'Total']
df[race_cols] = df[race_cols].map("{:,.1%}".format)
df[race_cols]

Race,White alone,Black or African American alone,American Indian alone,Alaska Native alone,American Indian and Alaska Native tribes specified,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Two or More Races
Path Type,,,,,,,,,
Bus,63.3%,6.8%,0.6%,0.1%,0.1%,15.6%,0.7%,3.9%,8.8%
Commuter Rail,64.4%,7.4%,0.7%,0.1%,0.1%,12.6%,1.3%,5.1%,8.3%
Drive,65.2%,5.7%,0.7%,0.1%,0.1%,14.3%,0.9%,4.3%,8.8%
Ferry 1,69.6%,4.9%,0.6%,0.1%,0.1%,13.8%,0.5%,3.1%,7.5%
Ferry 2,73.3%,4.0%,0.6%,0.1%,0.1%,9.7%,0.8%,3.6%,8.0%
Light Rail,62.6%,6.4%,0.5%,0.1%,0.1%,17.0%,0.7%,4.0%,8.5%


In [45]:
df_trip = df_trip.merge(df_hh[['hhno','hhincome']], on=['hhno'], how='left')

### Income distribution by household income of rider

In [46]:
# Create bins for hhincome
bins = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000, 200000, np.inf]
labels = ['<20k', '20-40k', '40-60k', '60-80k', '80-100k', '100-120k', '120-140k', '140-160k', '160-180k', '180-200k', '>200k']
df_trip['Income Bin'] = pd.cut(df_trip['hhincome'], bins=bins, labels=labels, right=False)
df_trip['Income Bin'] = df_trip['Income Bin'].astype('str')

# Transit Submodes by Income
df = pd.pivot_table(df_trip[(df_trip['Path Type']!="Drive")], values='trexpfac', index='Path Type', columns='Income Bin', aggfunc="sum")

# display labels with no significant figs
df[labels] = df[labels].map("{:0,.0f}".format)
df[labels]

Income Bin,<20k,20-40k,40-60k,60-80k,80-100k,100-120k,120-140k,140-160k,160-180k,180-200k,>200k
Path Type,,,,,,,,,,,
Bus,"62,068","59,879","55,274","49,836","47,945","39,616","36,280","32,005","25,379","25,135","126,164"
Commuter Rail,245,463,843,997,"1,271","1,047",932,763,690,660,"2,340"
Ferry 1,159,219,320,381,412,363,340,338,308,259,"1,732"
Ferry 2,"1,352","2,128","3,125","3,794","3,714","3,306","3,323","2,890","2,339","2,284","10,978"
Light Rail,"31,105","33,613","35,785","35,888","36,991","30,915","29,272","26,431","21,782","21,503","112,155"


In [47]:
pd.pivot_table(
    df_trip[df_trip['Path Type'] != "Drive"],
    values='hhincome',
    index='Path Type',
    aggfunc="median"
).astype(int).map(lambda x: "${:,.0f}".format(x))

,hhincome
Path Type,
Bus,"$102,491"
Commuter Rail,"$124,421"
Ferry 1,"$152,413"
Ferry 2,"$132,135"
Light Rail,"$121,930"
